In [157]:
from data_pipeline import DataPipeline
import warnings
warnings.filterwarnings('ignore')

# 📚 Data Collection

In [158]:
data_pipeline = DataPipeline()

# Get data from the TSE website about the candidates and election results
data_pipeline.collect_data(2022, "candidates", redownload=False)
data_pipeline.collect_data(2022, "social_media", redownload=False)
data_pipeline.collect_data(2022, "voting_section", redownload=False)
data_pipeline.collect_data(2022, "ibge", redownload=False)

2022 candidates files already downloaded. Skipping... ⏩
2022 social_media files already downloaded. Skipping... ⏩
2022 voting_section files already downloaded. Skipping... ⏩
2022 ibge files already downloaded. Skipping... ⏩


## Tables

### Voting section table
Information about voting places. Each row contains the following relevant infomation:
- Votes per voting section per round per votable entity: Column `"QT_VOTOS"`
- Voting section: Column `"NM_LOCAL_VOTACAO"`
- Round: Tells us if it's the first or second round, `"NR_TURNO"`
- Votable entity: This is not really a column, instead, it is a way to identify who/what we can vote for, which are:
  - People (candidates), which can be identified by the column `"SQ_CANDIDATO"`
  - Non-people, these can only be identified by the columns `"NR_VOTAVEL"` and `"DS_CARGO"`. The first tells us which one of the following entities are being votes for, the second column tells us for what position those votes are, which is important to differentiate Null votes for president and for governor, for example.
    - Null vote
    - White vote
    - Party vote
  
    So we can identify each votable entity by three columns: `"SQ_CANDIDATO"`, `"NR_VOTAVEL"` and `"DS_CARGO"`.

### Candidates table
Information about each candidate. For example, if people can vote for them (some candidates are excluded from the election before the voting day). Each row contains a different candidate.

### Social media table
Candidates' social media. Each row contains one social media link of a candidate, if they have multiple social media, they'll appear in multiple rows.

### Ibge table
Information about Brazilian states, like their population, population density, and Human Development Index (HDI)

# 🕵️‍♂️🛠️ Data Exploration and cleaning

In [159]:
data_pipeline.transform_to_sql_tables(directories=None)
# also edit data_pipeline so we can access the sql tables directly by doing data_pipeline.something

# Useful variables
UF = 'SC'
table_name = {
    'candidates': f'consulta_cand_2022_',
    'social_media': f'rede_social_candidato_2022_',
    'voting_section': f'votacao_secao_2022_'
}
total_votes_table_name = f'votacao_total_2022'
all_candidates_table_name = f'consulta_cand_2022'


Data transformation already done. ⏩


## Discovering voting section data

We'll start by exploring the table that contains information about the number of votes each candidate had, which is the most important information for us.

In [160]:

data_pipeline.execute_query(f'''
    SELECT "NM_VOTAVEL", "SQ_CANDIDATO", "NR_VOTAVEL", "QT_VOTOS", "NM_LOCAL_VOTACAO", "DS_CARGO", * 
    FROM public."{table_name["voting_section"]}{UF}" LIMIT 2
''')

,NM_VOTAVEL,SQ_CANDIDATO,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_CARGO,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,...,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
0,EDMILSON CARLOS PEREIRA JUNIOR,240001604520,4477,17,ESCOLA BÁSICA MUNICIPAL BRIGADEIRO EDUARDO GOMES,DEPUTADO FEDERAL,01/11/2022,16:05:25,2022,2,...,280,6,DEPUTADO FEDERAL,4477,EDMILSON CARLOS PEREIRA JUNIOR,17,1546,240001604520,ESCOLA BÁSICA MUNICIPAL BRIGADEIRO EDUARDO GOMES,"AVENIDA PEQUENO PRÍNCIPE, N. 2939"
1,EDMILSON CARLOS PEREIRA JUNIOR,240001604520,4477,3,CENTRO EDUCACIONAL MUNICIPAL RENASCER,DEPUTADO FEDERAL,01/11/2022,16:05:25,2022,2,...,188,6,DEPUTADO FEDERAL,4477,EDMILSON CARLOS PEREIRA JUNIOR,3,1147,240001604520,CENTRO EDUCACIONAL MUNICIPAL RENASCER,"RUA JOSÉ ANTÔNIO PEREIRA, S/N"


To calculate the total votes per votable entity (explained in detail earlier), we'll group the data by candidate ID ("SQ_CANDIDATO"), voting number ("NR_VOTAVEL), and political position ("DS_CARGO"), then sum up their "QT_VOTOS" column.

Next, we'll store the calculated total votes as "TOTAL_VOTOS," along with the three columns that identify the votable entities. This setup will allow us to perform JOIN operations with other tables.

## Creating table with total votes column from all states
Let's merge all voting section tables (there's one for each Brazilian state) into a single table.

In [169]:
query = f'''
CREATE TABLE IF NOT EXISTS public."{total_votes_table_name}" AS
SELECT * 
FROM (
'''

for UF in data_pipeline.UFs:
    query += f'''
    SELECT "SQ_CANDIDATO", "NR_VOTAVEL", SUM("QT_VOTOS") AS "TOTAL_VOTOS", "DS_CARGO", "SG_UF", "NR_TURNO"
    FROM public."{table_name["voting_section"]}{UF}"
    GROUP BY "SQ_CANDIDATO", "NR_VOTAVEL", "DS_CARGO", "SG_UF", "NR_TURNO"
    {f"UNION ALL" if UF != "TO" else ""} 
    '''

query += ') AS subquery_alias;'

# data_pipeline.execute_query(f'DROP TABLE IF EXISTS public."{total_votes_table_name}";', is_ddl=True)
data_pipeline.execute_query(query, is_ddl=True)

Below we can see the resulting table, with only the essential columns. Because of the GROUP BY performed, this new table with data from all Brazilian states has orders of magnitude less rows as than any individual voting section table.

In [170]:
data_pipeline.execute_query(f'SELECT * FROM public."{total_votes_table_name}" ORDER BY "TOTAL_VOTOS" DESC LIMIT 2')

,SQ_CANDIDATO,NR_VOTAVEL,TOTAL_VOTOS,DS_CARGO,SG_UF,NR_TURNO
0,250001615967,10,13480643.0,GOVERNADOR,SP,2
1,250001612465,13,10909371.0,GOVERNADOR,SP,2


## Discovering candidates data
Here we'll get only columns that can have some correlation with the number of votes a candidate had.

In [163]:
data_pipeline.execute_query(f'''
    SELECT "NM_CANDIDATO", "SQ_CANDIDATO", "NR_CANDIDATO", "DS_CARGO", * 
    FROM public."{table_name["candidates"]}{UF}" LIMIT 2
''')

,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,DS_CARGO,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,...,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS,ST_SUBSTITUIDO,SQ_SUBSTITUIDO,SQ_ORDEM_SUPLENCIA,DT_ACEITE_CANDIDATURA
0,JOSYANNE FENELON DA MOTA PEREIRA,270001700732,28888,DEPUTADO ESTADUAL,14/08/2023,19:31:51,2022,2,ELEIÇÃO ORDINÁRIA,1,...,#NULO#,NÃO,#NULO#,-1,#NULO#,N,N,-1,-1,14/08/2022 19:12:59
1,DENIS DIAS MENDES,270001700723,28555,DEPUTADO ESTADUAL,14/08/2023,19:31:51,2022,2,ELEIÇÃO ORDINÁRIA,1,...,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,-1,14/08/2022 19:12:59


The relevant columns are:
- "SQ_CANDIDATO": essential to perform JOIN's with other tables
- "NM_CANDIDATO": the candidate's name
- "NM_URNA_CANDIDATO": the candidate's name as it appeared on the voting machine
<!-- - "TP_AGREMIACAO": candidate party's coalition type -->
- "SG_PARTIDO": candidate party's acronym
- ...


## Creating table with candidates from all states

In [164]:
query = f'''
CREATE TABLE IF NOT EXISTS public."{all_candidates_table_name}" AS
SELECT * 
FROM (
'''

for UF in data_pipeline.UFs:
    query += f'''
    SELECT "SQ_CANDIDATO", "NR_VOTAVEL", SUM("QT_VOTOS") AS "TOTAL_VOTOS", "DS_CARGO", "SG_UF"
    FROM public."{table_name["candidates"]}{UF}"
    GROUP BY "SQ_CANDIDATO", "NR_VOTAVEL", "DS_CARGO", "SG_UF"
    {f"UNION ALL" if UF != "TO" else ""} 
    '''

query += ') AS subquery_alias;'

# data_pipeline.execute_query(f'DROP TABLE IF EXISTS public."{total_votes_table_name}";', is_ddl=True)
data_pipeline.execute_query(query, is_ddl=True)

An error occurred: column "NR_VOTAVEL" does not exist
LINE 6:     SELECT "SQ_CANDIDATO", "NR_VOTAVEL", SUM("QT_VOTOS") AS ...
                                   ^



## Merging created table with the candidates table

In [165]:
data_pipeline.execute_query(f'''
    SELECT *
    FROM "{total_votes_table_name}" as "table1"
    LEFT JOIN "{table_name["candidates"]}" AS "table2" ON "table1"."SQ_CANDIDATO" = "table2"."SQ_CANDIDATO" AND "table1"."NR_VOTAVEL" = "table2"."NR_CANDIDATO";
''')

An error occurred: relation "votacao_total_2022" does not exist
LINE 3:     FROM "votacao_total_2022" as "table1"
                 ^



In [166]:
data_pipeline.execute_query(f'SELECT * FROM public."{table_name["social_media"]}{UF}" LIMIT 2')

,DT_GERACAO,HH_GERACAO,AA_ELEICAO,SG_UF,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,SQ_CANDIDATO,NR_ORDEM,DS_URL
0,14/08/2023,19:33:24,2022,TO,2,ELEIÇÃO ORDINÁRIA,546,ELEIÇÕES GERAIS ESTADUAIS 2022,270001697336,1,https://dilma107778.com
1,14/08/2023,19:33:24,2022,TO,2,ELEIÇÃO ORDINÁRIA,546,ELEIÇÕES GERAIS ESTADUAIS 2022,270001700076,2,https://www.facebook.com/deputadoelenil


In [167]:
data_pipeline.execute_query(f'SELECT * FROM public."{table_name["candidates"]}{UF}" LIMIT 2')

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA,NM_TIPO_DESTINACAO_VOTOS,CD_SITUACAO_CANDIDATO_TOT,DS_SITUACAO_CANDIDATO_TOT,ST_PREST_CONTAS,ST_SUBSTITUIDO,SQ_SUBSTITUIDO,SQ_ORDEM_SUPLENCIA,DT_ACEITE_CANDIDATURA
0,14/08/2023,19:31:51,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,...,#NULO#,NÃO,#NULO#,-1,#NULO#,N,N,-1,-1,14/08/2022 19:12:59
1,14/08/2023,19:31:51,2022,2,ELEIÇÃO ORDINÁRIA,1,546,Eleições Gerais Estaduais 2022,02/10/2022,ESTADUAL,...,DEFERIDO,SIM,Válido,2,Deferido,S,N,-1,-1,14/08/2022 19:12:59


In [168]:
data_pipeline.execute_query("SELECT * FROM public.\"estados_brasil\" LIMIT 2")

,UF,Codigo,Gentilico,Governador,Capital,Area Territorial - km2,Populacao - pessoas,Densidade demografica - hab/km2,Matriculas no ensino fundamental - matriculas,IDH Indice de desenvolvimento humano,Receitas realizadas - R$ (x1000),Despesas empenhadas - R$ (x1000),Rendimento mensal domiciliar per capita - R$,Total de veiculos - veiculos,Unnamed: 14
0,Acre,12,acriano,GLADSON DE LIMA CAMELI,Rio Branco,164173.429,830026,5.06,153015,0.710,6.632883e+06,6.084417e+06,1038,334377,None
1,Alagoas,27,alagoano,PAULO SURUAGY DO AMARAL DANTAS,Macei&oacute;,27830.661,3127511,112.38,458782,0.684,1.195044e+07,1.046063e+07,935,1034187,None
